In [ ]:
# Part 1: Core LTR Pipeline
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
import lightgbm as lgb
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

def minimal_log_step(title):
    print(f"\n🔹 {title}")

def show_final_test_results(ndcg_results):
    print("\n✅ FINAL TEST RESULTS")
    print("=" * 30)
    for key, val in ndcg_results['avg_ndcg_scores'].items():
        print(f"{key.upper():<8}: {val:.4f} ({val * 100:.2f}%)")
    print("=" * 30)

# Load and merge data
def load_data():
    file1 = pd.read_excel("Simay_1.xlsx")
    file2 = pd.read_excel("Simay_2.xlsx")

    '''
    NEW DATA LOADING EXAMPLE

    new_file = pd.read_excel("new_data.xlsx") # write your file's name instead of new_data.xlsx
    file2 = pd.concat([file2, new_file], ignore_index=True)

    '''
    return pd.merge(file2, file1, on='CD_ORDERTYPE', how='left')

# Feature engineering - encoding and datetime processing
def process_data(df):
    df['DT_ORDER'] = pd.to_datetime(df['DT_ORDER'])
    df['SW_URGENTORDER'] = df['SW_URGENTORDER'].fillna(0)
    df['SW_CB_URGENTORDER'] = df['SW_CB_URGENTORDER'].fillna(0)
    df['IS_URGENT'] = ((df['SW_URGENTORDER'] == 1) | (df['SW_CB_URGENTORDER'] == 1)).astype(int)

    max_p = df['NO_PRIORITY'].max()
    df['FINAL_PRIORITY'] = df['IS_URGENT'] * 1000 + (max_p - df['NO_PRIORITY'] + 1)

    df['HOUR_OF_DAY'] = df['DT_ORDER'].dt.hour
    df['DAY_OF_WEEK'] = df['DT_ORDER'].dt.dayofweek
    df['ORDER_AGE_HOURS'] = (datetime.now() - df['DT_ORDER']).dt.total_seconds() / 3600

    le_dealer = LabelEncoder()
    le_ordertype = LabelEncoder()
    df['CD_DEALER_ENCODED'] = le_dealer.fit_transform(df['CD_DEALER'].astype(str))
    df['CD_ORDERTYPE_ENCODED'] = le_ordertype.fit_transform(df['CD_ORDERTYPE'].astype(str))

    return df, le_dealer, le_ordertype

# create_ltr_dataset function with business-driven scoring
def create_business_driven_ltr_dataset(df, min_group_size=4, max_group_size=50):
    grouped_data = []

    def normalize(col):
        col_min, col_max = col.min(), col.max()
        if col_max > col_min:
            return (col - col_min) / (col_max - col_min)
        else:
            return pd.Series(0.5, index=col.index)  # maintain index

    for part_id, group in df.groupby('CD_PART'):
        if len(group) < min_group_size or len(group) > max_group_size:
            continue

        group = group.copy()

        norm_priority = 1 - normalize(group['NO_PRIORITY'])
        norm_age = normalize(group['ORDER_AGE_HOURS'])
        norm_qty = normalize(group['QT_ORDER'])
        norm_hour = normalize(group['HOUR_OF_DAY'])

        noise = pd.Series(np.random.normal(0, 0.01, len(group)), index=group.index)

        group['BUSINESS_SCORE'] = (
            norm_priority * 0.4 +
            norm_age * 0.3 +
            norm_qty * 0.2 +
            norm_hour * 0.1 +
            noise
        )

        try:
            group['RELEVANCE_LABEL'] = pd.qcut(
                group['BUSINESS_SCORE'],
                q=5,
                labels=False,
                duplicates='drop'
            ).astype(int)
        except ValueError:
            continue

        if group['RELEVANCE_LABEL'].nunique() < 2:
            continue

        group['GROUP_ID'] = f"part_{part_id}"
        grouped_data.append(group)

    if not grouped_data:
        return None, None

    final_df = pd.concat(grouped_data, ignore_index=True)
    group_sizes = final_df.groupby('GROUP_ID').size().tolist()

    return final_df, group_sizes

'''
# Create robust LTR dataset
def create_ltr_dataset(df, min_group_size=4, max_group_size=50):
    groups = []
    for part_id, group in df.groupby('CD_PART'):
        if len(group) < min_group_size or len(group) > max_group_size:
            continue
        if group['FINAL_PRIORITY'].nunique() < 2:
            continue
        group = group.sort_values('FINAL_PRIORITY', ascending=False).copy()
        group['RELEVANCE_LABEL'] = pd.qcut(
            group['FINAL_PRIORITY'] + np.random.normal(0, 0.001, len(group)),
            q=min(5, len(group)),
            labels=False,
            duplicates='drop'
        )
        group['RELEVANCE_LABEL'] = group['RELEVANCE_LABEL'].max() - group['RELEVANCE_LABEL']
        group['GROUP_ID'] = f"part_{part_id}"
        if group['RELEVANCE_LABEL'].nunique() >= 2:
            groups.append(group)

    if not groups:
        return None, None
    final_df = pd.concat(groups, ignore_index=True)
    group_sizes = final_df.groupby('GROUP_ID').size().tolist()
    return final_df, group_sizes
'''

#EXCLUDE IS_URGENT from features
def prepare_features(df):
    cols = [
        'QT_ORDER', 'NO_PRIORITY',
        'HOUR_OF_DAY', 'DAY_OF_WEEK', 'ORDER_AGE_HOURS',
        'CD_DEALER_ENCODED', 'CD_ORDERTYPE_ENCODED'
    ]
    X = df[cols].copy()
    y = df['RELEVANCE_LABEL']

    num_cols = ['QT_ORDER', 'ORDER_AGE_HOURS', 'NO_PRIORITY']
    X[num_cols] = X[num_cols].astype(float)

    scaler = StandardScaler()
    X[num_cols] = scaler.fit_transform(X[num_cols])

    return X, y, cols, scaler

'''
# Feature prep
def prepare_features(df):
    cols = ['QT_ORDER', 'NO_PRIORITY', 'IS_URGENT',
            'HOUR_OF_DAY', 'DAY_OF_WEEK', 'ORDER_AGE_HOURS',
            'CD_DEALER_ENCODED', 'CD_ORDERTYPE_ENCODED']
    X = df[cols].copy()
    y = df['RELEVANCE_LABEL']
    num_cols = ['QT_ORDER', 'ORDER_AGE_HOURS', 'NO_PRIORITY']
    X[num_cols] = X[num_cols].astype(float)
    scaler = StandardScaler()
    X[num_cols] = scaler.fit_transform(X[num_cols])
    return X, y, cols, scaler
'''
# Split groups into train, validation, and test sets -not splitting orders-
def split_data(df, group_sizes, test_size=0.2, val_size=0.2):
    groups = df['GROUP_ID'].unique()
    train_g, temp_g = train_test_split(groups, test_size=test_size + val_size, random_state=42)
    val_g, test_g = train_test_split(temp_g, test_size=test_size / (test_size + val_size), random_state=42)

    train_df = df[df['GROUP_ID'].isin(train_g)]
    val_df = df[df['GROUP_ID'].isin(val_g)]
    test_df = df[df['GROUP_ID'].isin(test_g)]

    return (train_df, val_df, test_df), (
        train_df.groupby('GROUP_ID').size().tolist(),
        val_df.groupby('GROUP_ID').size().tolist(),
        test_df.groupby('GROUP_ID').size().tolist()
    )

# LightGBM training
def train_model(X_train, y_train, train_groups, X_val, y_val, val_groups):
    train_data = lgb.Dataset(X_train, label=y_train, group=train_groups)
    val_data = lgb.Dataset(X_val, label=y_val, group=val_groups, reference=train_data)
    params = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.05,
        'lambda_l1': 1.0,
        'lambda_l2': 1.0,
        'min_data_in_leaf': 10,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'max_depth': 6,
        'verbose': -1,
        'ndcg_eval_at': [1, 3, 5, 10],
    }
    model = lgb.train(
        params,
        train_data,
        valid_sets=[val_data],
        num_boost_round=1000,
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
    )
    return model

# NDCG evaluation
def evaluate_model(model, X_test, y_test, test_groups):
    y_pred = model.predict(X_test)
    start = 0
    scores = {'ndcg_1': [], 'ndcg_3': [], 'ndcg_5': [], 'ndcg_10': []}
    for size in test_groups:
        end = start + size
        y_true = np.array([y_test[start:end]])
        y_score = np.array([y_pred[start:end]])
        scores['ndcg_1'].append(ndcg_score(y_true, y_score, k=1))
        scores['ndcg_3'].append(ndcg_score(y_true, y_score, k=3))
        scores['ndcg_5'].append(ndcg_score(y_true, y_score, k=5))
        scores['ndcg_10'].append(ndcg_score(y_true, y_score, k=min(10, size)))
        start = end
    avg_scores = {k: np.mean(v) for k, v in scores.items()}
    return {'avg_ndcg_scores': avg_scores}

# Part 2: Detailed Analysis Module
def run_detailed_analyses(df, ltr_df, model, feature_columns, le_dealer, le_ordertype, X_test, y_test, test_groups):
    print("\n📊 RUNNING DETAILED ANALYSES")
    print("=" * 50)

    # 1. Feature Importance
    print("\n🔍 Feature Importance")
    importance_scores = model.feature_importance(importance_type='gain')
    feature_mapping = {
        'QT_ORDER': 'Order Quantity',
        'NO_PRIORITY': 'Rule-Based Priority Level',
        'IS_URGENT': 'Urgent Flag',
        'HOUR_OF_DAY': 'Hour of Day',
        'DAY_OF_WEEK': 'Day of Week',
        'ORDER_AGE_HOURS': 'Order Age',
        'CD_DEALER_ENCODED': 'Dealer Type',
        'CD_ORDERTYPE_ENCODED': 'Order Type'
    }
    feature_importance_df = pd.DataFrame({
        'technical': feature_columns,
        'label': [feature_mapping.get(col, col) for col in feature_columns],
        'importance': importance_scores
    })
    feature_importance_df['%'] = (
        feature_importance_df['importance'] / feature_importance_df['importance'].sum()
    ) * 100
    feature_importance_df = feature_importance_df.sort_values('%', ascending=False)

    for _, row in feature_importance_df.iterrows():
        print(f"• {row['label']:<22}: {row['%']:.2f}%")

    # 2. Rule-based vs. LTR ranking
    print("\n⚖️  Rule-Based vs. LTR Ranking Differences")
    changes = []
    for part in ltr_df['CD_PART'].unique():
        part_df = ltr_df[ltr_df['CD_PART'] == part].copy()
        if len(part_df) < 2:
            continue
        part_df = part_df.sort_values("FINAL_PRIORITY", ascending=False).reset_index(drop=True)
        part_df['RULE_RANK'] = range(1, len(part_df)+1)

        X_part = part_df[feature_columns]
        part_df['LTR_SCORE'] = model.predict(X_part)
        part_df = part_df.sort_values("LTR_SCORE", ascending=False).reset_index(drop=True)
        part_df['LTR_RANK'] = range(1, len(part_df)+1)

        part_df['DIFF'] = part_df['RULE_RANK'] - part_df['LTR_RANK']
        if part_df['DIFF'].abs().sum() > 0:
            changes.append(part_df['DIFF'].abs().mean())

    if changes:
        print(f"• Average rank difference in changed parts: {np.mean(changes):.2f}")
        print(f"• % of parts with changed rankings        : {len(changes)/ltr_df['CD_PART'].nunique()*100:.1f}%")
    else:
        print("• No significant differences between rule-based and LTR rankings.")

    # 3. Temporal Dynamics
    print("\n⏳ Temporal Priority Dynamics")
    df['ORDER_MONTH'] = df['DT_ORDER'].dt.month
    urgency_by_month = df.groupby('ORDER_MONTH')['IS_URGENT'].mean() * 100
    for month, pct in urgency_by_month.items():
        print(f"• Month {month:>2}: {pct:5.1f}% urgent orders")

    # 4. Executive Summary
    print("\n📋 EXECUTIVE SUMMARY")
    print("=" * 50)
    top_feat = feature_importance_df.iloc[0]
    print(f"• Top feature driving decisions: {top_feat['label']} ({top_feat['%']:.1f}%)")
    if changes:
        print(f"• {len(changes)} parts showed altered rankings by LTR")
    else:
        print(f"• LTR aligns closely with rule-based for all parts")
    print(f"• Urgency peaks in month: {urgency_by_month.idxmax()} ({urgency_by_month.max():.1f}%)")

    return {
        'feature_importance': feature_importance_df,
        'avg_rank_diff': np.mean(changes) if changes else 0,
        'urgency_by_month': urgency_by_month
    }

# Entry point
def main_pipeline(run_analysis=False):
    minimal_log_step("Loading Data")
    df = load_data()

    minimal_log_step("Processing Data")
    df, le_dealer, le_ordertype = process_data(df)

    minimal_log_step("Creating LTR Dataset")
    ltr_df, group_sizes = create_business_driven_ltr_dataset(df)
    #ltr_df, group_sizes = create_ltr_dataset(df)

    if ltr_df is None:
        print("❌ No valid groups found for LTR training.")
        return

    minimal_log_step("Splitting Data")
    (train_df, val_df, test_df), (train_groups, val_groups, test_groups) = split_data(ltr_df, group_sizes)

    minimal_log_step("Preparing Features")
    X_train, y_train, feature_columns, scaler = prepare_features(train_df)
    X_val, y_val, _, _ = prepare_features(val_df)
    X_test, y_test, _, _ = prepare_features(test_df)

    minimal_log_step("Training Model")
    model = train_model(X_train, y_train, train_groups, X_val, y_val, val_groups)

    minimal_log_step("Evaluating NDCG Scores")
    ndcg_results = evaluate_model(model, X_test, y_test, test_groups)
    show_final_test_results(ndcg_results)

    return {
        'df': df,
        'ltr_df': ltr_df,
        'model': model,
        'feature_columns': feature_columns,
        'le_dealer': le_dealer,
        'le_ordertype': le_ordertype,
        'X_test': X_test,
        'y_test': y_test,
        'test_groups': test_groups
    } if run_analysis else None

result = main_pipeline(run_analysis=False)
if result:
    run_detailed_analyses(
        df=result['df'],
        ltr_df=result['ltr_df'],
        model=result['model'],
        feature_columns=result['feature_columns'],
        le_dealer=result['le_dealer'],
        le_ordertype=result['le_ordertype'],
        X_test=result['X_test'],
        y_test=result['y_test'],
        test_groups=result['test_groups']
    )


🔹 Loading Data

🔹 Processing Data

🔹 Creating LTR Dataset

🔹 Splitting Data

🔹 Preparing Features

🔹 Training Model
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[250]	valid_0's ndcg@1: 0.914476	valid_0's ndcg@3: 0.914816	valid_0's ndcg@5: 0.928875	valid_0's ndcg@10: 0.946661

🔹 Evaluating NDCG Scores

✅ FINAL TEST RESULTS
NDCG_1  : 0.9493 (94.93%)
NDCG_3  : 0.9346 (93.46%)
NDCG_5  : 0.9458 (94.58%)
NDCG_10 : 0.9590 (95.90%)
